# 获取中国金融期货交易所（中金所）数据

In [1]:
import os
from pathlib import Path

NOTEBOOK_PATH: Path = Path(os.path.abspath(''))
PATH_FOR_RESULT: Path = NOTEBOOK_PATH.parent.joinpath('RESULT')

## 获取历史数据

[中金所官网](http://www.cffex.com.cn/)顶部菜单“数据” -> “历史数据下载”。

按照上期所的方式观察，可以发现，中金所数据是按月提供的。

In [2]:
from pathlib import Path
import datetime as dt

import requests


def download_cffex_history_data(save_path: Path, year: int, month: int) -> None:
    """
    下载中国金融期货交易所（中金所，CFFEX）的历史数据。
    :param save_path: 保存的位置。
    :param year: 需要下载数据的年份。
    :param month: 需要下载数据的月份。
    :return: None.
    """
    
    # 中金所历史数据 url 模板。
    url: str = 'http://www.cffex.com.cn/sj/historysj/{year:4d}{month:02d}/zip/{year:4d}{month:02d}.zip'

    # 中金所历史数据从 2010 年 4 月（股指期货）开始提供。
    start_year: int = 2010
    start_month: int = 4
    today: dt.date = dt.date.today()

    # 如果参数 <year> 和 <month> 不在合理范围，引发异常。
    if month < 1 or month > 12:
        raise ValueError(f'参数 <month> 取值范围在 [1, 12]。')
    if year < start_year or (year == start_year and month < start_month):
        raise ValueError(f'中金所历史数据自{start_year:4d}年{start_month:02d}月起开始提供。')
    if year > today.year or (year == today.year and month > today.month):
        raise ValueError(f'{year:4d}年{month:02d}月是未来日期。')

    # 如果参数 <save_path> 不存在，引发异常。
    if not save_path.exists():
        raise FileNotFoundError(f'目录 {save_path} 不存在。')

    # 下载。
    response = requests.get(url.format(year=year, month=month))
    
    # 如果下载不顺利，引发异常。
    if response.status_code != 200:
        raise requests.exceptions.HTTPError(
            f'下载 <{url.format(year=year, month=month)}> 时发生错误。'
        )

    # 保存文件。
    with open(save_path.joinpath(f'CFFEX_{year:4d}-{month:02d}.zip'), 'wb') as f:
        f.write(response.content)

# 测试。
download_cffex_history_data(
    save_path=PATH_FOR_RESULT,
    year=2020,
    month=8
)

自动化下载从2010年04月（股指期货上市，中金所开始提供数据的时间）到现在的所有历史数据：

In [3]:
def download_all_cffex_history_data(save_path: Path) -> None:
    """
    下载中国金融期货交易所（中金所，CFFEX）的全部历史数据。
    :param save_path: 保存的位置。
    :return: None.
    """
    start_year: int = 2010
    start_month: int = 4
    today: dt.date = dt.date.today()
    
    for year in range(start_year, today.year + 1):
        for month in range(1, 12 + 1):
            if year == 2010 and month < start_month:
                continue
            if year == today.year and month > today.month:
                break
            download_cffex_history_data(save_path=save_path, year=year, month=month)

# 测试。
download_all_cffex_history_data(PATH_FOR_RESULT)

Done.